# Daya Bay model in Jupyter notebook

## Preparation

Make sure that you run Jupyter server from root directory of the project.

## Note

The next code is needed to set the proper current working directory for loading model data.
```python
import os

os.chdir("../")
```
If you put notebook to root directory of the project, please remove these lines.

In [ ]:
%matplotlib

In [ ]:
import os

os.chdir("../")

from dagflow.tools.logger import DEBUG as INFO4
from dagflow.tools.logger import INFO1, INFO2, INFO3, set_level

from models import load_model
from dgf_statistics.minimizer.iminuitminimizer import IMinuitMinimizer



set_level(INFO1)

## Model parameters

`MODEL_*` variables describe possible parameters of the loaded model. To get more info about parameters, just check documentation of `load_model`.

## Next sample

Method `next_sample()` is called to generate a new sample of pseudo data.

In [ ]:
MODEL_VERSION = "v0"
MODEL_SOURCE_TYPE = "npz"
MODEL_ANUE_SPEC = "exponential"
MODEL_SEED = 0
MODEL_MC_DATA = "asimov"
CONCATENATION_MODE = "detector"

model = load_model(
    MODEL_VERSION,
    source_type=MODEL_SOURCE_TYPE,
    spectrum_correction_mode=MODEL_ANUE_SPEC,
    monte_carlo_mode=MODEL_MC_DATA,
    seed=MODEL_SEED,
    concatenation_mode=CONCATENATION_MODE,
)

model.next_sample()

storage = model.storage
parameters_free = storage("parameters.free")
parameters_constrained = storage("parameters.constrained")
statistic = storage("outputs.statistic")

In [ ]:
chi2_type = "stat.chi2p"

parameters_oscprob = dict(parameters_constrained["oscprob"].walkjoineditems())
parameters_oscprob.update(dict(parameters_free("oscprob").walkjoineditems()))
parameters_oscprob_gn = parameters_oscprob.copy()
parameters_oscprob_gn.update(dict(parameters_free("detector").walkjoineditems()))
parameters_oscprob_spec = parameters_oscprob.copy()
parameters_oscprob_spec.update(dict(parameters_free("neutrino_per_fission_factor").walkjoineditems()))

In [ ]:
chi2 = statistic[chi2_type]

minimizer_gn = IMinuitMinimizer(chi2, parameters=parameters_oscprob_gn)
minimizer_spec = IMinuitMinimizer(chi2, parameters=parameters_oscprob_spec)

In [ ]:
minimizer_gn.push_initial_values()
minimizer_gn.fit()

In [ ]:
minimizer_spec.push_initial_values()
minimizer_spec.fit()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def visualization(args):
    plt.plot(model.storage.get_value("outputs.eventscount.final.concatenated.selected").data, label="Model")
    data = model.storage.get_value("outputs.data.pseudo.self").data
    x = np.arange(data.shape[0])
    plt.errorbar(x=x, y=data, xerr=0.5, yerr=data**.5, label="DATA", linestyle="none")
    plt.legend()

minimizer_gn._minimizer.interactive(plot=visualization)

In [ ]:
minimizer_spec._minimizer.interactive(plot=visualization)